In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,precision_recall_fscore_support
import warnings
import os

In [ ]:
df1= pd.read_excel("/kaggle/input/credit-risk-ds/case_study1.xlsx")
df2=pd.read_excel("/kaggle/input/credit-risk-ds/case_study2.xlsx")

In [ ]:
df1.head()

In [ ]:
df1.head()

In [ ]:
print(df1.shape)
print(df2.shape)

## Data Cleaning

In [ ]:
## Remove the null values

df1=df1.loc[df1['Age_Oldest_TL']!= -99999]

* By have doimne knlowedge we can consider that -99999 is nothing but has null values so we can remove the -99999 values from the Age_Oldest_TL column in realtime we can discuss with business owner take the information about column and values we can remove the null values

In [ ]:
## After removing the null values
df1.shape

* In df2 data we have -99999 in almost all the column so we can set a thresold that 10000 means we can iterate each and every column find if the --99999 values has more than 10000 we can remove thos rows

In [ ]:
column_to_removed = []
for i in df2.columns:
     if df2.loc[df2[i] == -99999].shape[0]>10000:
        column_to_removed.append(i)

In [ ]:
df2= df2.drop(column_to_removed,axis=1)

In [ ]:
df2.shape

In [ ]:
for i in df2.columns:
    df2=df2.loc[df2[i] != -99999]

In [ ]:
df2.shape

In [ ]:
## Merge the both dataset check the common coulnm in the data set
for i in list(df1.columns):
    if i in list(df2.columns):
            print(i)

In [ ]:
df = pd.merge(df1,df2, how= 'inner',left_on = ['PROSPECTID'],right_on=['PROSPECTID'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
## Check the categeorical columns
df_cat =df.select_dtypes(include='O')
df_cat.columns

In [ ]:
for i in df_cat:
    print(df[i].value_counts())

In [ ]:
## Now the check the categeorical columns are  how much corelated to target column
## Lable encoding is done only when data is odinal in nature like data values like HIGH ,LOW MEDAIUM we can do label en coding but in the MARITALSTATUS if we give 0 for singal and 1 for married this not give much meaning so we can chiq square test

for i in df_cat.columns:
    chi2,pval,_ , _ = chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i, "---",pval)

* Since all the categeorical  features  have pval< 0.05  we accept all the columns

In [ ]:
#df_num =df.select_dtypes(exclude='O')
#df_num.columns

numerical_col = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numerical_col.append(i)
        
numerical_col

In [ ]:
len(numerical_col)

In [ ]:
vif_data =df[numerical_col]
total_columns =vif_data.shape[1]
columns_to_be_kept= []
column_index =0

In [ ]:
vif_data

In [ ]:
for i in range(0,total_columns):
    vif_value = variance_inflation_factor(vif_data,column_index)
    print(column_index,'----',vif_value)
    
    if vif_value <=6:
        columns_to_be_kept.append(numerical_col[i])
        column_index =column_index +1
    else:
        vif_data=vif_data.drop([numerical_col[i]],axis=1)
        

In [ ]:
print(columns_to_be_kept)
print('Total columns after the vif : ',len(columns_to_be_kept))

In [ ]:
## Check the anova test with target variable
from scipy.stats import f_oneway
columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a =list(df[i])
    b =  list(df['Approved_Flag'])
    
    group_P1 = [value for value ,group in zip(a,b) if group == 'P1']
    group_P2 = [value for value ,group in zip(a,b) if group == 'P2']
    group_P3 = [value for value ,group in zip(a,b) if group == 'P3']
    group_P4 = [value for value ,group in zip(a,b) if group == 'P4']
    
    f_statistic,p_value = f_oneway(group_P1,group_P2,group_P3,group_P4)
    
    if p_value <=0.05:
        columns_to_be_kept_numerical.append(i)
    

In [ ]:
print(columns_to_be_kept_numerical)
print("Total columns after anova : ",len(columns_to_be_kept_numerical))

In [ ]:
## Combine all the features 
features= columns_to_be_kept_numerical + ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']
df= df[features + ['Approved_Flag']]


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
## make copy of clean data 
df1=df.copy(deep=True)


In [ ]:
## Export the cleaned data into csv file
df1.to_csv('/kaggle/working/cleaned.csv', index=False)

In [ ]:
df1.shape

In [ ]:
##Covert categeorical varibale into numerical
for i in df_cat:
    print(df1[i].unique())

In [ ]:
df1['EDUCATION']=df1['EDUCATION'].map({'12TH':2,'GRADUATE':3,'SSC':1,'POST-GRADUATE':4,'UNDER GRADUATE':3,'OTHERS':1,'PROFESSIONAL':3})
df1['EDUCATION']

In [ ]:
df1['EDUCATION'].value_counts()

In [ ]:
df1.info()

In [ ]:
df_encoded =pd.get_dummies(df1,columns =['MARITALSTATUS','GENDER','last_prod_enq2','first_prod_enq2'],dtype=int)
df_encoded

In [ ]:
df_encoded.info()

In [ ]:
df_encoded.describe()

## Model Training

In [ ]:
X= df_encoded.drop(['Approved_Flag'],axis=1)
y= df_encoded['Approved_Flag']

In [ ]:
y.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
rfc=RandomForestClassifier(n_estimators= 200,random_state =42)
rfc.fit(X_train,y_train)

In [ ]:
#y_pred=rfc.predict(X_test)
y_pred_test=rfc.predict(X_test)
y_pred_train=rfc.predict(X_train)
Test_accuracy  = accuracy_score(y_test,y_pred_test)
Train_accuracy  = accuracy_score(y_train,y_pred_train)

print(f'Test Accuracy:{Test_accuracy}')
print(f'Train Accuracy:{Train_accuracy}')
print()

In [ ]:
precision,recall,f1_score ,_ = precision_recall_fscore_support(y_test,y_pred_test)

for i ,v in enumerate (['p1','p2','p3','p4']):
    print(f"Class: {v} " )
    print(f"Precision :{precision[i]} " )
    print(f"Recall: {recall[i]} " )
    print(f"F1 Score: {f1_score[i]} " )
    print()
        

## XGBoost

In [ ]:
X= df_encoded.drop(['Approved_Flag'],axis=1)
y= df_encoded['Approved_Flag']


In [ ]:
from  sklearn.preprocessing import LabelEncoder
label_encoder =LabelEncoder()
y_encoder =label_encoder.fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X,y_encoder,test_size=0.2,random_state=42)

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
#y_pred=xgb.predict(X_test)

In [ ]:
y_pred_test=xgb.predict(X_test)
y_pred_train=xgb.predict(X_train)
Test_accuracy  = accuracy_score(y_test,y_pred_test)
Train_accuracy  = accuracy_score(y_train,y_pred_train)

print(f'Test Accuracy:{Test_accuracy}')
print(f'Train Accuracy:{Train_accuracy}')
print()

In [ ]:
precision,recall,f1_score ,_ = precision_recall_fscore_support(y_test,y_pred_test)

for i ,v in enumerate (['p1','p2','p3','p4']):
    print(f"Class: {v} " )
    print(f"Precision :{precision[i]} " )
    print(f"Recall: {recall[i]} " )
    print(f"F1 Score: {f1_score[i]} " )
    print()
        


## Decesion Tree

In [ ]:
X= df_encoded.drop(['Approved_Flag'],axis=1)
y= df_encoded['Approved_Flag']
X_train,X_test,y_train,y_test = train_test_split(X,y_encoder,test_size=0.2,random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(max_depth = 20 ,min_samples_split =10)
dtc.fit(X_train,y_train)


In [ ]:
y_pred_test=dtc.predict(X_test)
y_pred_train=dtc.predict(X_train)
Test_accuracy  = accuracy_score(y_test,y_pred_test)
Train_accuracy  = accuracy_score(y_train,y_pred_train)

print(f'Test Accuracy:{Test_accuracy}')
print(f'Train Accuracy:{Train_accuracy}')
print()

In [ ]:
precision,recall,f1_score ,_ = precision_recall_fscore_support(y_test,y_pred_test)

for i ,v in enumerate (['p1','p2','p3','p4']):
    print(f"Class: {v} " )
    print(f"Precision :{precision[i]} " )
    print(f"Recall: {recall[i]} " )
    print(f"F1 Score: {f1_score[i]} " )
    print()
        

## By comparing 3 models we can say Xgboost is performing better and we apply some fine tunning to that model

In [ ]:
## check the data is balanced or imblanced 
df_encoded['Approved_Flag'].value_counts()

* By looking the target valriable we seen that the data  is not much skew so it is balanced 

In [ ]:
## Hyperparamater Tunning

from sklearn.model_selection import GridSearchCV
X_train,X_test,y_train,y_test = train_test_split(X,y_encoder,test_size=0.2,random_state=42)
xgb=XGBClassifier(objective='multi:softmax',num_class =4)
param_grid ={
    #'colsample_bytree': [0.1,0.3,0.5,0.7,0.9],
    'learning_rate': [0.01,0.1,0.2],
    'max_depth': [5,7,10],
    #'alpha' : [1,101,100],
    'n_estimators' :[10,50,100,200]
    
}
grid_search=GridSearchCV(estimator=xgb,param_grid=param_grid,cv=3,scoring ='accuracy',n_jobs=-1)
grid_search.fit(X_train,y_train)


In [ ]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
xgb_class=XGBClassifier(learning_rate = 0.2, max_depth = 2, n_estimators = 200)
xgb_class.fit(X_train,y_train)

y_pred_test=xgb_class.predict(X_test)
y_pred_train=xgb_class.predict(X_train)
Test_accuracy  = accuracy_score(y_test,y_pred_test)
Train_accuracy  = accuracy_score(y_train,y_pred_train)

print(f'Test Accuracy:{Test_accuracy}')
print(f'Train Accuracy:{Train_accuracy}')
print()

In [ ]:
precision,recall,f1_score ,_ = precision_recall_fscore_support(y_test,y_pred_test)

for i ,v in enumerate (['p1','p2','p3','p4']):
    print(f"Class: {v} " )
    print(f"Precision :{precision[i]} " )
    print(f"Recall: {recall[i]} " )
    print(f"F1 Score: {f1_score[i]} " )
    print()

In [ ]:
import pickle
with open('/kaggle/working/Credit_Risk_Modeling','wb') as f:
    pickle.dump(xgb_class,f)